In [27]:
#!pip install gymnasium
#!pip install numpy

In [28]:
import gymnasium as gym
import numpy as np
import random

In [29]:
env = gym.make('Taxi-v3', render_mode = "rgb_array")


In [30]:
state = env.reset()

print(state)

(22, {'prob': 1.0, 'action_mask': array([1, 0, 0, 1, 0, 0], dtype=int8)})


In [31]:
num_steps = 99
for s in range(num_steps+1):

    print(f"step: {s} out of {num_steps}")

    # sample a random action from the list of available actions
    action = env.action_space.sample()

    # perform this action on the environment
    env.step(action)

    # print the new state
    env.render()

# end this instance of the taxi environment
env.close()

step: 0 out of 99
step: 1 out of 99
step: 2 out of 99
step: 3 out of 99
step: 4 out of 99
step: 5 out of 99
step: 6 out of 99
step: 7 out of 99
step: 8 out of 99
step: 9 out of 99
step: 10 out of 99
step: 11 out of 99
step: 12 out of 99
step: 13 out of 99
step: 14 out of 99
step: 15 out of 99
step: 16 out of 99
step: 17 out of 99
step: 18 out of 99
step: 19 out of 99
step: 20 out of 99
step: 21 out of 99
step: 22 out of 99
step: 23 out of 99
step: 24 out of 99
step: 25 out of 99
step: 26 out of 99
step: 27 out of 99
step: 28 out of 99
step: 29 out of 99
step: 30 out of 99
step: 31 out of 99
step: 32 out of 99
step: 33 out of 99
step: 34 out of 99
step: 35 out of 99
step: 36 out of 99
step: 37 out of 99
step: 38 out of 99
step: 39 out of 99
step: 40 out of 99
step: 41 out of 99
step: 42 out of 99
step: 43 out of 99
step: 44 out of 99
step: 45 out of 99
step: 46 out of 99
step: 47 out of 99
step: 48 out of 99
step: 49 out of 99
step: 50 out of 99
step: 51 out of 99
step: 52 out of 99
ste

In [32]:
state_size = env.observation_space.n  # total number of states (S)
action_size = env.action_space.n      # total number of actions (A)

# initialize a qtable with 0's for all Q-values
qtable = np.zeros((state_size, action_size))

print(f"Q table:")
print(f"{qtable}")

Q table:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [33]:
# hyperparameters to tune
learning_rate = 0.9
discount_rate = 0.8

# dummy variables
reward = 10 # R_(t+1)
state = env.observation_space.sample()      # S_t
action = env.action_space.sample()          # A_t
new_state = env.observation_space.sample()  # S_(t+1)

# Qlearning algorithm: Q(s,a) := Q(s,a) + learning_rate * (reward + discount_rate * max Q(s',a') - Q(s,a))
qtable[state, action] += learning_rate * (reward + discount_rate * np.max(qtable[new_state,:]) - qtable[state,action])

print(f"Q-value for (state, action) pair ({state}, {action}): {qtable[state,action]}")

Q-value for (state, action) pair (26, 0): 9.0


In [34]:
# dummy variables
episode = random.randint(0,500)
qtable = np.random.randn(env.observation_space.sample(), env.action_space.sample())

# hyperparameters
epsilon = 1.0     # probability that our agent will explore
decay_rate = 0.01 # of epsilon

if random.uniform(0,1) < epsilon:
    # explore
    action = env.action_space.sample()
else:
    # exploit
    action = np.argmax(qtable[state,:])

# epsilon decreases exponentially --> our agent will explore less and less
epsilon = np.exp(-decay_rate*episode)

In [35]:
import gym
import numpy as np
import random

# Define color codes for console output
class bcolors:
    RED = '\u001b[31m'
    GREEN = '\u001b[32m'
    RESET = '\u001b[0m'

# Check Gym version
print(f"Gym Version: {gym.__version__}")

# Create Taxi environment with 'ansi' render mode
env = gym.make('Taxi-v3', render_mode='ansi')

# Initialize Q-table
state_size = env.observation_space.n
action_size = env.action_space.n
qtable = np.zeros((state_size, action_size))

# Hyperparameters
learning_rate = 0.9
discount_rate = 0.8
epsilon = 1.0
decay_rate = 0.005

# Training variables
num_episodes = 2000
max_steps = 99  # per episode

print("AGENT IS TRAINING...")

for episode in range(num_episodes):
    # Reset the environment
    result = env.reset()
    if len(result) == 2:
        state, _ = result
    else:
        state = result

    done = False

    for step in range(max_steps):
        # Exploration-exploitation tradeoff
        if random.uniform(0, 1) < epsilon:
            # Explore: select a random action
            action = env.action_space.sample()
        else:
            # Exploit: select the action with max value (best known action)
            action = np.argmax(qtable[state, :])

        # Take an action and observe the outcome
        result = env.step(action)

        # Adjust unpacking based on Gym version
        if len(result) == 5:
            new_state, reward, terminated, truncated, info = result
            done = terminated or truncated
        else:
            new_state, reward, done, info = result

        # Q-learning update
        qtable[state, action] = qtable[state, action] + learning_rate * (
            reward + discount_rate * np.max(qtable[new_state, :]) - qtable[state, action]
        )

        # Transition to the new state
        state = new_state

        # If done, finish the episode
        if done:
            break

    # Decay epsilon
    epsilon = np.exp(-decay_rate * episode)

# Display the trained Q-table
print(f"Our Q-table:\n{qtable}")
print(f"Training completed over {num_episodes} episodes")
input("Press Enter to see our trained taxi agent")

# Number of episodes to preview
episodes_to_preview = 3

for episode in range(episodes_to_preview):
    # Reset the environment
    result = env.reset()
    if len(result) == 2:
        state, _ = result
    else:
        state = result

    done = False
    episode_rewards = 0

    print(f"+++++ EPISODE {episode + 1} +++++")

    for step in range(max_steps):
        print(f"TRAINED AGENT")
        print(f"Step {step + 1}")

        # Exploit: select the best action based on Q-table
        action = np.argmax(qtable[state, :])

        # Take the action and observe the outcome
        result = env.step(action)

        # Adjust unpacking based on Gym version
        if len(result) == 5:
            new_state, reward, terminated, truncated, info = result
            done = terminated or truncated
        else:
            new_state, reward, done, info = result

        # Accumulate rewards
        episode_rewards += reward

        # Render the environment in 'ansi' mode and print
        rendered = env.render()
        print(rendered)
        print("")  # For better readability

        # Display the score with appropriate color
        if episode_rewards < 0:
            print(f"Score: {bcolors.RED}{episode_rewards}{bcolors.RESET}")
        else:
            print(f"Score: {bcolors.GREEN}{episode_rewards}{bcolors.RESET}") 

        # Transition to the new state
        state = new_state

        # If done, finish the episode
        if done:
            print("Episode finished!")
            break  

# Close the Taxi environment
env.close()


Gym Version: 0.26.2
AGENT IS TRAINING...
Our Q-table:
[[  0.           0.           0.           0.           0.
    0.        ]
 [ -4.09224253  -2.3506629   -4.04827134  -4.00562406  -1.6445568
  -12.82913406]
 [  0.24284197   1.54918439   0.20729709   1.50612674   3.192
   -7.44644673]
 ...
 [ -2.7393912   -2.694834    -2.7454428    1.51444792 -11.36129098
  -10.7118    ]
 [ -3.15611648  -2.94150269  -3.21894773  -0.82692343 -11.99598356
  -12.31795001]
 [ -1.638       -1.647       -0.999       14.99999697  -9.
    0.6818399 ]]
Training completed over 2000 episodes
+++++ EPISODE 1 +++++
TRAINED AGENT
Step 1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


Score: -1
TRAINED AGENT
Step 2
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


Score: -2
TRAINED AGENT
Step 3
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


Score: -3
TRAINED AGENT
Step 4
+--